In [ ]:
# Importing necessary libraries and modules
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from utils import to_datetime
import matplotlib.pyplot as plt

In [ ]:
# Load data
countries = pd.read_csv("./data/village_savings/Countries.csv")
locations = pd.read_csv("./data/village_savings/Locations.csv")
regions = pd.read_csv("./data/village_savings/Regions.csv")
members = pd.read_csv("./data/village_savings/Members.csv", sep="\t") #tab seperated csv
group_members = pd.read_csv("./data/village_savings/GroupMembers.csv")
groups = pd.read_csv("./data/village_savings/SavingsGroups.csv", sep="\t")


In [ ]:
# Join data: Location Name, Region Name, and Country Name
groups = groups.merge(locations, on="LocationId").merge(regions, on="RegionId").merge(countries, on="CountryCode")

In [ ]:
groups.head()

In [ ]:
# Get Member of groups
# groups[groups.GroupStatusCode != 'E']
groups['NumberOfMembers'] = 0
for idx, row in groups.iterrows():
   if row.IsKhusaServiceOffered == 1:
       groups.at[idx, 'NumberOfMembers'] = group_members[lambda x: x.GroupID == row.GroupID]['GroupID'].count()

In [ ]:
# Create pivot table using RegionName and CountryName columns.
group_pivot_table = pd.pivot_table(groups[groups.NumberOfMembers > 0], values='NumberOfMembers', index=['CountryName', 'RegionName'], columns=['GroupName'], fill_value=0)

In [ ]:
#Question 1. How many groups and members are there currently registered? And what is their geographical coverage? (Country and Region)
sns.heatmap(group_pivot_table, annot=True,  cmap="YlGnBu")

In [ ]:
group_members.head()

In [ ]:
# group_members = group_members.merge(members[['MemberID', 'Gender']], on="MemberID").merge(groups[['GroupID', 'GroupName', 'RegionName']], on="GroupID")
group_members = group_members.merge(members[['MemberID', 'Gender']], on="MemberID", how='left').merge(groups[['GroupID', 'GroupName', 'RegionName']], on="GroupID" , how="left")

In [ ]:
# pd.crosstab([group_members.GroupName],group_members.Gender).plot.bar()
group_members.groupby(group_members.GroupName).Gender.value_counts(normalize=True).unstack().plot.bar(stacked=True)

In [ ]:
group_members.groupby([group_members.RegionName, group_members.Gender]).Gender.value_counts(normalize=True).unstack().plot.bar(stacked=True)


In [ ]:
major_cities = ['Lilongwe', 'Zomba', 'Mzuzu', 'Blantyre'] #major cities in Malawi
# groups[groups.LocationName.isin(major_cities)]
groups[groups.CountryName == 'Malawi'][groups.LocationName.isin(major_cities)].groupby(['LocationName']).NumberOfMembers.sum().plot.pie()

In [ ]:
group_members.head()

In [ ]:
group_members.DateCreated = group_members.DateCreated.apply(lambda x: to_datetime(x, "%Y-%m-%d %H:%M:%S.%f")) #convert column to to_datetime

In [24]:
# group_members.groupby([group_members.DateCreated.dt.year]).MemberID.count().max().sort_by()
groups.head()
# group_members.groupby([group_members.DateCreated.dt.month, group_members.DateCreated.dt.year]).MemberID.max()



,GroupID,GroupName,MaximumMemberCount,LanguageCode,CurrencyCode,LocationId,IsActive,GroupStatusCode,GroupTypeCode,DateCreated,LocationName,RegionId,RegionName,CountryCode,CountryName,PhoneCode,IsKhusaServiceOffered,GMTHours,NumberOfMembers
0,1,Brakus-Borer,20,en-GB,MWK,7,1,A,V,20:09.8,Lilongwe,2,Central,MW,Malawi,265,1,2,19
1,2,"Bergstrom, Raynor and Gutmann",10,en-GB,MWK,7,1,A,V,31:19.9,Lilongwe,2,Central,MW,Malawi,265,1,2,0
2,3,Carroll and Sons,8,en-GB,MWK,7,1,E,V,29:02.8,Lilongwe,2,Central,MW,Malawi,265,1,2,0
3,4,Hoppe-Lueilwitz,10,en-GB,MWK,7,1,E,M,04:19.3,Lilongwe,2,Central,MW,Malawi,265,1,2,0
4,8,Ward LLC,20,en-GB,MWK,7,1,E,V,05:46.1,Lilongwe,2,Central,MW,Malawi,265,1,2,1
